In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import tensorflow as tf
from tracker import Object_Tracker
from ultralytics import YOLO
import random
from deep_sort import generate_detections as gdet


In [2]:
tracker= Object_Tracker()

In [3]:
model = YOLO('yolov8n.pt')

In [4]:
data_dir= 'M:\\dataset2014\\results'

In [5]:
all_items = os.listdir(data_dir)
categories = [item for item in all_items if os.path.isdir(os.path.join(data_dir, item))]
print(categories)

['badWeather', 'baseline', 'cameraJitter', 'dynamicBackground', 'intermittentObjectMotion', 'lowFramerate', 'nightVideos', 'PTZ', 'shadow', 'thermal', 'turbulence']


In [6]:
img=  'M:\\dataset2014\\results\\badWeather\\skating\\in000001.jpg'
img= cv2.imread(img)
results= model(img)

names=results[0].names


0: 640x640 1 car, 534.0ms
Speed: 6.0ms preprocess, 534.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


In [7]:
img.shape

(320, 320, 3)

In [8]:
colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]

detection_threshold = 0.5

In [9]:
pwd

'C:\\Users\\Toqa Alaa\\deep_learning\\An-Application-for-Moving-Object-Removal-in-Unlabeled-Image-Databases\\Object_tracking\\ObjectTracking-DeepSORT-YOLOv3-TF2\\An-Application-for-survelience-camera-tracking'

In [10]:
data_path='M:\\dataset2014\\results\\baseline\\highway'
video_path= 'test.mp4'

In [11]:
# model_filename = 'model_data/mars-small128.pb'
# encoder = gdet.create_box_encoder(model_filename, batch_size=1)

In [12]:
vid = cv2.VideoCapture(0) 

In [19]:
def non_max_suppression(detections,  threshold):

    if len(detections) == 0:
        return [], [], []

    boxes= [box[0:4] for box in detections]
    scores= [box[4].item() for box in detections]
    classes= [box[5] for box in detections]
    
    boxes= np.array(boxes)
    scores= np.array(scores)
    classes= np.array(classes)

    # Initialize an empty list to store the selected boxes.
    selected_boxes = []
    selected_scores= []
    selected_classes= []

    # Sort the boxes by their confidence scores in descending order.
    sorted_indices = np.argsort(scores)[::-1]
    boxes = boxes[sorted_indices]
    scores = scores[sorted_indices]
    classes= classes[sorted_indices]

    while len(boxes) > 0:
        # Select the box with the highest confidence score and add it to the selected list.
        selected_boxes.append(boxes[0])
        selected_scores.append(scores[0])
        selected_classes.append(classes[0])

        # Calculate the IoU (Intersection over Union) between the selected box and the remaining boxes.
        iou = calculate_iou(selected_boxes[-1], boxes[1:])

        # Filter out the boxes with IoU greater than or equal to the threshold.
        mask = iou < threshold
        boxes = boxes[1:][mask]
        scores = scores[1:][mask]

    return selected_boxes, selected_scores, selected_classes

In [20]:
def calculate_iou(box1, boxes2):
    x1, y1, x2, y2 = box1
    x1 = np.maximum(x1, boxes2[:, 0])
    y1 = np.maximum(y1, boxes2[:, 1])
    x2 = np.minimum(x2, boxes2[:, 2])
    y2 = np.minimum(y2, boxes2[:, 3])

    intersection_area = np.maximum(0, x2 - x1 + 1) * np.maximum(0, y2 - y1 + 1)
    box1_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    boxes2_area = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)

    iou = intersection_area / (box1_area + boxes2_area - intersection_area)
    return iou

In [25]:
size = (320,320)
out = cv2.VideoWriter('project.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for img in os.listdir(data_path):
    frame = cv2.imread(os.path.join(data_path, img))
# while(True):
        
#     _, frame = vid.read()
    # Perform object detection using YOLO on the current frame
    results = model(frame)  

    # Update the tracks using the DeepSort tracker
    for result in results:
        detections = []
        

        # Creating the detections array
        for r in result.boxes.data:
            x1, y1, x2, y2, score, class_id = r
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)

            class_id = int(class_id)
            class_name= names[class_id]
            if score>=0.35:
                detections.append([x1, y1, x2, y2, score, class_name])
            
        
       
        boxes, score, classes= non_max_suppression(detections, 0.7)
        detections=[]
        for i in range(len(boxes)):
            b= []
            b.append(boxes[i][0])
            b.append(boxes[i][1])
            b.append(boxes[i][2])
            b.append(boxes[i][3])
            b.append(score[i])
            b.append(classes[i])
            
            
            detections.append(b)
        
#         print(detections)
    
        tracker.update(frame, detections)
        # Updating the tracks with the new frame
        for track in tracker.tracks:
            bbox = track.bbox
            x1, y1, x2, y2 = bbox
            track_id = track.track_id

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 3)

            # Define the text you want to add
            text = names[class_id]

            # Define the position where you want to place the text (adjust coordinates as needed)
            text_x = int(x1)  # You can adjust the x-coordinate as needed
            text_y = int(y1) - 10  # You can adjust the y-coordinate as needed

            # Get the color of the rectangle
            rectangle_color = colors[track_id % len(colors)]

            # Get the size of the text to calculate the size of the background rectangle
            (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            background_width = text_width + 10
            background_height = text_height + 10

            # Define the position of the background rectangle
            background_x = text_x
            background_y = text_y - text_height

            # Draw the background rectangle
            cv2.rectangle(frame, (background_x, background_y), (background_x + background_width, background_y + background_height), rectangle_color, -1)

            # Define the font and font scale
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5

            # Define the color of the text (you can change this if needed)
            text_color = (255, 255, 255)  # White color in BGR

            # Add the text to the frame
            cv2.putText(frame, text, (text_x, text_y), font, font_scale, text_color, 1, cv2.LINE_AA)
        # Save the frame with tracking results
        cv2.imshow('img', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()


0: 640x640 1 car, 516.0ms
Speed: 4.0ms preprocess, 516.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)



2


0: 640x640 2 cars, 538.0ms
Speed: 6.0ms preprocess, 538.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)



2
2


0: 640x640 1 car, 536.0ms
Speed: 4.0ms preprocess, 536.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)



...

0: 640x640 3 cars, 1 truck, 541.0ms
Speed: 5.0ms preprocess, 541.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)



2
2
2
7


0: 640x640 3 cars, 1 truck, 538.0ms
Speed: 4.0ms preprocess, 538.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)



2
2
2
7


KeyboardInterrupt: 